In [4]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType
from pyspark.sql import functions as f

StatementMeta(tcccibele, 12, 4, Finished, Available)

In [5]:
# Definir schema do arquivo
schema = StructType([ 
    StructField("Ano", StringType(), True), 
    StructField("Trimestre", StringType(), True), 
    StructField("Mes", StringType(), True), 
    StructField("Dia", StringType(), True), 
    StructField("Pais", StringType(), True), 
    StructField("UF", StringType(), True), 
    StructField("GrupoVulneravel", StringType(), True), 
    StructField("CanalAtendimento", StringType(), True), 
    StructField("CenarioViolacao", StringType(), True), 
    StructField("Frequencia", StringType(), True), 
    StructField("Motivacao", StringType(), True), 
    StructField("Denunciante", StringType(), True), 
    StructField("SexoVitima", StringType(), True),
    StructField("VitimaLGBT", StringType(), True),
    StructField("RacaVitima", StringType(), True),
    StructField("EscolaridadeVitima", StringType(), True),
    StructField("IdadeVitima", StringType(), True),
    StructField("FaixaRendaVitima", StringType(), True),
    StructField("ProfissaoVitima", StringType(), True),
    StructField("ReligiaoVitima", StringType(), True),
    StructField("DeficienciaVitima", StringType(), True),
    StructField("DeficienciaDoencaRara", StringType(), True),
    StructField("Categoria1", StringType(), True),
    StructField("Categoria2", StringType(), True),
    StructField("Categoria3", StringType(), True),
    StructField("Categoria4", StringType(), True),
    StructField("Categoria5", StringType(), True),
    StructField("Categoria6", StringType(), True)
])

StatementMeta(tcccibele, 12, 5, Finished, Available)

In [6]:
# Ler arquivo de origem da landing zone
df = spark.read.format("csv") \
            .schema(schema) \
            .options(quote='"', sep=';', encoding='ISO-8859-1') \
            .load('/landing_zone/2020/segundo_semestre_2020.csv')

StatementMeta(tcccibele, 12, 6, Finished, Available)

In [8]:
# Remover colunas e linhas sem valor para o data lake
df = df.where((df[0].isNotNull()) \
                & (df[0] != 'Ano'))

StatementMeta(tcccibele, 12, 8, Finished, Available)

In [9]:
# Capitalizar todos os textos e incluir 0 onde há nulos
names = df.schema.names
for name in names:
    df = df.withColumn(name, f.initcap(f.col(name)))
    #df = df.withColumn(name,f.regexp_replace(name,',',''))
    #df = df.withColumn(name, df[name].cast(IntegerType()))

df = df.fillna(0)

StatementMeta(tcccibele, 12, 9, Finished, Available)

In [10]:
df.limit(3).toPandas()

StatementMeta(tcccibele, 12, 10, Finished, Available)

,Ano,Trimestre,Mes,Dia,Pais,UF,GrupoVulneravel,CanalAtendimento,CenarioViolacao,Frequencia,Motivacao,Denunciante,SexoVitima,VitimaLGBT,RacaVitima,EscolaridadeVitima,IdadeVitima,FaixaRendaVitima,ProfissaoVitima,ReligiaoVitima,DeficienciaVitima,DeficienciaDoencaRara,Categoria1,Categoria2,Categoria3,Categoria4,Categoria5,Categoria6
0,2020,Trim 4,Dezembro,31,Brasil,Am,02. Violência Doméstica E Familiar Contra A Mu...,Telefônico,Casa Onde Reside A Vítima E O Suspeito,Única Ocorrência,Em Razão De Ser Mulher,A Própria Vítima,Feminino,*(n/d),Parda,Ensino Médio Completo,35 A 39 Anos,Até 1 Sm,*(n/d),Evangelho Quadrangular,*(n/d),Não,Integridade,Física,Exposição De Risco À Saúde,None,None,None
1,2020,Trim 4,Dezembro,31,Brasil,Am,02. Violência Doméstica E Familiar Contra A Mu...,Telefônico,Casa Onde Reside A Vítima E O Suspeito,Única Ocorrência,*(n/d),A Própria Vítima,Feminino,*(n/d),Parda,Ensino Médio Completo,35 A 39 Anos,Até 1 Sm,*(n/d),Evangelho Quadrangular,*(n/d),Não,Integridade,Física,Lesão Corporal,None,None,None
2,2020,Trim 4,Dezembro,31,Brasil,Am,02. Violência Doméstica E Familiar Contra A Mu...,Telefônico,Casa Onde Reside A Vítima E O Suspeito,Única Ocorrência,Por Condutas Excessivas/ Desnecessárias/ Desac...,A Própria Vítima,Feminino,*(n/d),Parda,Ensino Médio Completo,35 A 39 Anos,Até 1 Sm,*(n/d),Evangelho Quadrangular,Não Tem Deficiência,Não,Integridade,Física,Agressão Ou Vias De Fato,None,None,None


In [11]:
# Escrever dados limpos em formato parque no azure storage
#df.write.format('parquet').mode('overwrite').save('/history_zone/2020/PrimeiroSemestre/')

df.write.partitionBy("Ano","Mes").mode("overwrite").parquet("/history_zone/2020/SegundoSemestre/")


StatementMeta(tcccibele, 12, 11, Finished, Available)

In [13]:
df = spark.read.parquet('/history_zone/2020/SegundoSemestre/')
df.limit(3).toPandas()

StatementMeta(tcccibele, 12, 13, Finished, Available)

,Trimestre,Dia,Pais,UF,GrupoVulneravel,CanalAtendimento,CenarioViolacao,Frequencia,Motivacao,Denunciante,SexoVitima,VitimaLGBT,RacaVitima,EscolaridadeVitima,IdadeVitima,FaixaRendaVitima,ProfissaoVitima,ReligiaoVitima,DeficienciaVitima,DeficienciaDoencaRara,Categoria1,Categoria2,Categoria3,Categoria4,Categoria5,Categoria6,Ano,Mes
0,Trim 4,31,Brasil,Mg,04. Violência Contra Pessoa Idosa,Telefônico,Casa Da Vítima,Diariamente,Em Razão Da Idade,Anônimo,Feminino,*(n/d),Branca,*(n/d),90+,Até 1 Sm,*(n/d),*(n/d),Não Tem Deficiência,Não,Integridade,Física,Agressão Ou Vias De Fato,None,None,None,2020,Dezembro
1,Trim 4,31,Brasil,Mg,04. Violência Contra Pessoa Idosa,Telefônico,Casa Da Vítima,Única Ocorrência,Com Humilhação,A Própria Vítima,Feminino,*(n/d),Preta,Ensino Médio Completo,60 A 64 Anos,Até 1 Sm,*(n/d),Católica Apostólica Romana,*(n/d),Não,Integridade,Psíquica,Assédio Moral,Injúria,None,None,2020,Dezembro
2,Trim 4,31,Brasil,Mg,04. Violência Contra Pessoa Idosa,Telefônico,Casa Da Vítima,Única Ocorrência,Em Razão Do Sexo Biológico,A Própria Vítima,Feminino,*(n/d),Preta,Ensino Médio Completo,60 A 64 Anos,Até 1 Sm,*(n/d),Católica Apostólica Romana,*(n/d),Não,Integridade,Psíquica,Bullying,None,None,None,2020,Dezembro
